# Imputing data

### Support Functions

In [14]:
import numpy as np
import matplotlib.pyplot as plt
    
def make_features_labels(dataset, feature_names, remove_nan=False):
    """
    Quick way to split a dataset into features and labels based on feature names
    """
    data = featureFormat(dataset, feature_names, 
                         sort_keys=True, remove_NaN=remove_nan)
    labels, features = targetFeatureSplit(data)
    labels = np.array(labels)
    features = np.array(features)
    
    return features, labels

### Get necessary data and variables from [feature_selection](feature_selection.ipynb)

In [15]:
import pickle
with open('data/final_project_dataset.pkl', 'rb') as f:
    fin_data = pickle.load(f)

# Remove 'TOTAL' from data
del fin_data['TOTAL']

# Convert negative numbers in data to zero
count = 0
for key1, item in fin_data.items():
    for key2 in item.keys():
        if item[key2] < 0:
            item[key2] = 0
            count += 1
print '%i negative values changed to zero' % count

with open('selected_feature_names.pkl', 'rb') as f:
    selected_feature_names = pickle.load(f)
    
selected_features, selected_labels = make_features_labels(fin_data, selected_feature_names)

66 negative values changed to zero


In [31]:
selected_features[:,1]

array([  4.48444200e+06,   1.82466000e+05,   9.16197000e+05,
         5.63434300e+06,   8.27696000e+05,   8.60136000e+05,
         9.69068000e+05,   5.50163000e+06,   1.02500000e+05,
         2.28474000e+05,   6.18850000e+05,   1.54562900e+07,
         2.04759300e+06,   2.01483500e+06,   1.27900000e+03,
         2.66958900e+06,   4.92880000e+04,   1.05463700e+06,
         1.27158200e+06,   2.35570200e+06,   1.63929700e+06,
         4.77557000e+05,   1.86875800e+06,              nan,
                    nan,              nan,   1.49034400e+06,
                    nan,   1.10139300e+06,   8.07956000e+05,
         1.20864900e+06,   4.74797900e+06,   5.50981000e+05,
         1.20458300e+06,   1.41046400e+06,   2.36815100e+06,
         3.19941000e+05,   8.75760000e+05,   7.74920000e+04,
         2.09326300e+06,   2.69232400e+06,   2.11725000e+05,
         3.67634000e+06,   2.42408300e+06,   1.41485700e+06,
                    nan,   1.81755000e+05,   1.72525300e+07,
         5.05910000e+04,

In [35]:
'nan' in selected_features[:,1]

/Users/TRN/anaconda/envs/ml27/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


False

In [18]:
from sklearn.preprocessing import Imputer
imp = Imputer()
new_features = imp.fit_transform(selected_features)

In [19]:
new_features

array([[  2.01955000e+05,   4.48444200e+06,   1.72954100e+06, ...,
          1.52000000e+02,   3.04805000e+05,   4.70000000e+01],
       [  2.84087543e+05,   1.82466000e+05,   2.57817000e+05, ...,
          4.65276663e+05,   7.46491200e+05,   6.48953488e+01],
       [  4.77000000e+02,   9.16197000e+05,   4.04615700e+06, ...,
          8.64523000e+05,   7.46491200e+05,   3.90000000e+01],
       ..., 
       [  2.84087543e+05,   2.62342118e+06,   1.39130000e+05, ...,
          4.65276663e+05,   7.46491200e+05,   6.48953488e+01],
       [  1.58403000e+05,   3.60300000e+05,   8.30855200e+06, ...,
          1.47950000e+05,   7.46491200e+05,   6.48953488e+01],
       [  2.84087543e+05,   5.50970000e+04,   1.92758000e+05, ...,
          4.65276663e+05,   7.46491200e+05,   6.48953488e+01]])